In [1]:
import pathlib
import pandas as pd
import glob
import os
import seaborn as sns
import numpy as np
import pathlib as pl
from stmetric.utils import (get_sol_filepaths, 
                            get_sol_instrument, 
                            get_unique_instruments, 
                            get_sol_IMT, 
                            get_fname, 
                            get_sol_pitch_dynamics,
                            df_attach_meta)

SOL_PATH = '/import/c4dm-datasets/SOL_0.9_HQ/'

meta_path = '/homes/cv300/Documents/timbre-metric/jasmp/'
SEED_FILELIST = os.path.join(meta_path, 'seed_filelist.csv')
SEED_FILELIST_PITCHED = os.path.join(meta_path, 'seed_filelist_pitched.csv')

EXT_FILELIST_PITCHED = os.path.join(meta_path, 'extended_filelist_pitched.csv')
EXT_FILELIST = os.path.join(meta_path, 'extended_filelist.csv')
SIM = os.path.join(meta_path, 'sim.csv')
SIM_PITCHED = os.path.join(meta_path, 'sim_pitched.csv')
INST_TO_FAMILY = os.path.join(meta_path, 'instrument_to_family.csv')

df = pd.read_csv(SEED_FILELIST, index_col=0)
sim = pd.read_csv(SIM, header=None).to_numpy(dtype=np.float32)

inst_to_fam = pd.read_csv(INST_TO_FAMILY, index_col=0).set_index('instrument').to_dict()['family']
df_ext = pd.read_csv(EXT_FILELIST, index_col=0)
sim = pd.read_csv(SIM, header=None).to_numpy(dtype=np.float32)

### get seed filepaths and instrument families

In [2]:
fname_to_path = {os.path.splitext(os.path.basename(f))[0]: f 
                 for f in get_sol_filepaths(SOL_PATH, list(df['fname']))}
df['fpath'] = [fname_to_path[f] for f in list(df['fname'])]

df['instrument family'] = [inst_to_fam[pl.Path(fname_to_path[f]).parts[0]] 
                           for f in list(df['fname'])]

### delete sounds without a pitch

In [3]:
pd.set_option('display.max_rows', 78)
unpitched = [i for i, f in enumerate(list(df['fname'])) 
             if not get_sol_pitch_dynamics(f)]
pitch_dynamics = [get_sol_pitch_dynamics(f) for i, f in enumerate(list(df['fname'])) 
                  if i not in unpitched]

for idx in unpitched:
    df = df.drop(df.index[idx])
    sim = np.delete(sim, idx, axis=0)
    sim = np.delete(sim, idx, axis=1)
df['pitch'] = [i[0] for i in pitch_dynamics]
df['dynamics'] = [i[1] for i in pitch_dynamics]

# np.save('/homes/cv300/Documents/timbre-metric/jasmp/sim_pitched.npy', sim)
# df.to_csv('/homes/cv300/Documents/timbre-metric/jasmp/seed_filelist_pitched.csv')

### get extended filelist in pitch range

In [11]:
from stmetric.utils import get_extended_files, filter_pitch

df_ext = get_extended_files(filter_fn=filter_pitch, semitones=48)
df_ext

,seed_id,filepath,instrument,IMT,pitch,dynamics,instrument family
0,8,Horns/Horn/ordinario/Hn-ord-B4-ff.wav,Hn,Hn-ord,B4,ff,Brass
1,8,Horns/Horn/ordinario/Hn-ord-D#5-pp.wav,Hn,Hn-ord,D#5,pp,Brass
2,8,Horns/Horn/ordinario/Hn-ord-C3-mf.wav,Hn,Hn-ord,C3,mf,Brass
3,8,Horns/Horn/ordinario/Hn-ord-B3-ff.wav,Hn,Hn-ord,B3,ff,Brass
4,8,Horns/Horn/ordinario/Hn-ord-C2-pp.wav,Hn,Hn-ord,C2,pp,Brass
...,...,...,...,...,...,...,...
7236,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-E1-mf-1...,BBTb,BBTb-slap,E1,mf,Brass
7237,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-F1-f.wav,BBTb,BBTb-slap,F1,f,Brass
7238,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-D1-mf-2...,BBTb,BBTb-slap,D1,mf,Brass
7239,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-D#1-mf-...,BBTb,BBTb-slap,D#1,mf,Brass


In [12]:
df_ext.to_csv('/homes/cv300/Documents/timbre-metric/jasmp/extended_pitch_F4.csv')


### extended files dynamics only

In [1]:
from stmetric.utils import get_extended_files, filter_dynamics

df_ext = get_extended_files(filter_fn=filter_dynamics)
df_ext

,seed_id,filepath,instrument,IMT,pitch,dynamics,instrument family
0,8,Horns/Horn/ordinario/Hn-ord-C4-pp.wav,Hn,Hn-ord,C4,pp,Brass
1,8,Horns/Horn/ordinario/Hn-ord-C4-mf.wav,Hn,Hn-ord,C4,mf,Brass
2,8,Horns/Horn/ordinario/Hn-ord-C4-ff.wav,Hn,Hn-ord,C4,ff,Brass
3,9,Horns/Horn/slap-pitched/Hn-slap-C4-mf.wav,Hn,Hn-slap,C4,mf,Brass
4,30,Strings/Viola/pizzicato-bartok/Va-pizz-bartok-...,Va,Va-pizz-bartok,C4,ff-4c,Strings
...,...,...,...,...,...,...,...
175,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-F1-ff-1...,BBTb,BBTb-slap,F1,ff-1,Brass
176,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-F1-mf.wav,BBTb,BBTb-slap,F1,mf,Brass
177,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-F1-ff-2...,BBTb,BBTb-slap,F1,ff-2,Brass
178,57,Tubas/Bass-Tuba/slap-pitched/BBTb-slap-F1-f.wav,BBTb,BBTb-slap,F1,f,Brass


In [8]:
import pandas as pd
df_ext = pd.read_csv('/homes/cv300/Documents/timbre-metric/jasmp/extended_dynamics.csv', index_col=0)
df_seed = pd.read_csv(SEED_FILELIST, index_col=0)

In [24]:
ext_row = df_ext.iloc[0] 
seed_id = ext_row['seed_id']
seed_fname = df_seed.iloc[seed_id]['fname']
from stmetric.utils import get_sol_IMT
get_sol_IMT(seed_fname) == get_sol_IMT(os.path.basename(ext_row.filepath))

True

In [34]:
# df_ext_tmp[df_ext_tmp['filepath'].isin(ext_files)].to_csv("/homes/cv300/Documents/timbre-metric/jasmp/extended.csv")
df_ext_tmp.to_csv("/homes/cv300/Documents/timbre-metric/jasmp/extended_dynamics.csv")

In [89]:
fname_to_path = {os.path.splitext(os.path.basename(f))[0]: f 
                 for f in get_sol_filepaths(SOL_PATH, list(df['fname']))}
df['fpath'] = [fname_to_path[f] for f in list(df['fname'])]

import pathlib as pl
df['instrument family'] = [inst_to_fam[pl.Path(fname_to_path[f]).parts[0]] 
                           for f in list(df['fname'])]

,fname,instrument,IMT,pitch,dynamics,fpath,instrument family,num_extended
0,Hp-xyl-C4-p,Hp,Hp-xyl,C4,p,PluckedStrings/Harp/xylophonic/Hp-xyl-C4-p.wav,Brass,69
1,ASax-key-cl-C4-p,ASax,ASax-key-cl,C4,p,Saxophones/Alto-Sax/key-click/ASax-key-cl-C4-p...,Brass,12
2,ASax-slap-C4-mf,ASax,ASax-slap,C4,mf,Saxophones/Alto-Sax/slap-pitched/ASax-slap-C4-...,Strings,64
3,Gtr-ord-hi-reg-E5-mf-3c,Gtr,Gtr-ord-hi-reg,E5,mf-3c,PluckedStrings/Guitar/ordinario-high-register/...,Strings,30
4,Gtr-pizz-C4-mf-2c,Gtr,Gtr-pizz,C4,mf-2c,PluckedStrings/Guitar/pizzicato/Gtr-pizz-C4-mf...,Strings,59
...,...,...,...,...,...,...,...,...
73,Ob-ord-C4-mf,Ob,Ob-ord,C4,mf,Oboes/Oboe/ordinario/Ob-ord-C4-mf.wav,Brass,209
74,Vn-nonvib-C4-mf-4c,Vn,Vn-nonvib,C4,mf-4c,Strings/Violin/non-vibrato/Vn-nonvib-C4-mf-4c.wav,Brass,244
75,Vc-legno-tratto-C4-mf-1c,Vc,Vc-legno-tratto,C4,mf-1c,Strings/Violoncello/col-legno-tratto/Vc-legno-...,Brass,45
76,Vc-nonvib-C4-mf-1c,Vc,Vc-nonvib,C4,mf-1c,Strings/Violoncello/non-vibrato/Vc-nonvib-C4-m...,Brass,278


In [ ]:
sim